<b>Привет!</b>

Меня зовут Харочкина Екатерина, и я буду проверять твой проект. Давай общаться на «ты», но если это неудобно, пожалуйста, сообщи. Моя основная цель — не указать на совершенные тобою ошибки, а поделиться своим опытом и помочь тебе совершенствоваться как профессионалу.

Ты подготовил замечательный проект, но есть аспекты, которые еще можно улучшить. Я буду использовать комментарии с цветовой разметкой. Пожалуйста, **не перемещай, не изменяй и не удаляй их**. Это поможет выполнить повторную проверку твоего проекта оперативнее.

<div class="alert alert-danger">
<b>Комментарий ревьюера №1</b> 
    
Таким комментарием выделены самые важные замечания. Без их отработки проект не будет принят.
</div>

<div class="alert alert-warning">
<b>Комментарий ревьюера №1</b> 
    
Таким комментарием выделены небольшие замечания и рекомендации, которые будет хорошо учесть.
</div>

<div class="alert alert-success">
<b>Комментарий ревьюера №1</b> 
    
Таким комментарием выделены удачные решения🙂
</div>


Давай работать над проектом в диалоге: если ты **что-то меняешь** в проекте по моим рекомендациям — **пиши об этом**. После того, как ты все доработаешь, я смогу принять твой проект!) 

# Исследование надежности заемщиков


Во второй части проекта вы выполните шаги 3 и 4. Их вручную проверит ревьюер.
Чтобы вам не пришлось писать код заново для шагов 1 и 2, мы добавили авторские решения в ячейки с кодом. 



## Откройте таблицу и изучите общую информацию о данных

**Задание 1. Импортируйте библиотеку pandas. Считайте данные из csv-файла в датафрейм и сохраните в переменную `data`. Путь к файлу:**

`/datasets/data.csv`

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

**Задание 2. Выведите первые 20 строчек датафрейма `data` на экран.**

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Задание 3. Выведите основную информацию о датафрейме с помощью метода `info()`.**

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

**Задание 4. Выведите количество пропущенных значений для каждого столбца. Используйте комбинацию двух методов.**

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**Задание 5. В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце вы обработаете на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.**

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

**Задание 6. В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработайте значения в этом столбце: замените все отрицательные значения положительными с помощью метода `abs()`.**

In [6]:
data['days_employed'] = data['days_employed'].abs()

**Задание 7. Для каждого типа занятости выведите медианное значение трудового стажа `days_employed` в днях.**

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

**Задание 8. Выведите перечень уникальных значений столбца `children`.**

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

**Задание 9. В столбце `children` есть два аномальных значения. Удалите строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**Задание 10. Ещё раз выведите перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

**Задание 11. Заполните пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**Задание 12. Убедитесь, что все пропуски заполнены. Проверьте себя и ещё раз выведите количество пропущенных значений для каждого столбца с помощью двух методов.**

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

**Задание 13. Замените вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

**Задание 14. Обработайте неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведите их к нижнему регистру. Проверьте остальные столбцы.**

In [14]:
data['education'] = data['education'].str.lower()

**Задание 15. Выведите на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалите их.**

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

**Задание 16. На основании диапазонов, указанных ниже, создайте в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используйте собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Задание 17. Выведите на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Задание 18. Создайте функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используйте собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучите данные в столбце `purpose` и определите, какие подстроки помогут вам правильно определить категорию.**

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследуйте данные и ответьте на вопросы

<div class="alert alert-danger">
<b>Комментарий ревьюера №1</b> Замечание по всему разделу ответов на вопросы
    
Гораздо рациональнее здесь воспользоваться `pivot_table()`.
    
Чтобы найти долю должников, нужно количество должников поделить на общее число заемщиков. Эту долю можно найти с помощью функции агрегации `mean`
    
Попробуй получить сводную следующего вида. В индексе столбец, который исследуем (в первом вопросе это количество детей), а столбцы: 'Всего кредитополучателей', 'Всего должников', 'Доля должников'. Учитывай, что использовать параметр columns не обязательно, а в параметр aggfunc можно указывать несколько функций агрегации сразу.

Также могу посоветовать переименовать столбцы и написать функцию для построения сводных во всех вопросах.
</div>

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [22]:
#child1 = data.groupby('children')['debt'].sum() # число должников
#child2 = data.groupby('children')['debt'].count() # число займов
#child_debt_mean = child1 / child2  # поиск конверсии
#print(child1)
#print(child2)
#print(child_debt_mean.apply('{:.2%}'.format)) 

child_debt_mean = data.pivot_table(index=['children'], values='debt', aggfunc=['count', 'sum', 'mean'])
child_debt_mean.columns = ['Всего кредитополучателей', 'Всего должников', 'Доля должников']
child_debt_mean['Доля должников'] = child_debt_mean['Доля должников']*100
child_debt_mean

# Ваш код будет здесь. Вы можете создавать новые ячейки.

,Всего кредитополучателей,Всего должников,Доля должников
children,,,
0,14091,1063,7.543822
1,4808,444,9.234609
2,2052,194,9.454191
3,330,27,8.181818
4,41,4,9.756098
5,9,0,0.000000


<div class="alert alert-warning">
<b>Комментарий ревьюера №2</b> 
    
Переменная data_pivot лишняя, лучше ее убрать
</div>

**Вывод:** Да, зависимость есть. Чем меньше детей - тем чаще возвращают кредит в срок. 5 детей сложно учитывать, так как данных крайне мало для такого анализа. 4 тоже не много, но вот 1-3 вполне подойдут.

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [26]:
#status1 = data.groupby('family_status')['debt'].sum() # число должников
#status2 = data.groupby('family_status')['debt'].count() # число займов
#family_status_debt_mean = status1 / status2  # поиск конверсии
#print(status1)
#print(status2)
#print(family_status_debt_mean.apply('{:.2%}'.format)) 

family_status_debt_mean  = data.pivot_table(index=['family_status'], values='debt', aggfunc=['count', 'sum', 'mean'])
family_status_debt_mean.columns = ['Всего кредитополучателей', 'Всего должников', 'Доля должников']
family_status_debt_mean['Доля должников'] = family_status_debt_mean['Доля должников']*100
family_status_debt_mean.sort_values(by = 'Доля должников', ascending = True)


# Ваш код будет здесь. Вы можете создавать новые ячейки.

,Всего кредитополучателей,Всего должников,Доля должников
family_status,,,
вдовец / вдова,951,63,6.624606
в разводе,1189,84,7.064760
женат / замужем,12261,927,7.560558
гражданский брак,4134,385,9.313014
Не женат / не замужем,2796,273,9.763948


**Вывод:** Да, зависимость есть. Как видно из показателей - женатые/замужние чаще выплачивают займы, чем не женатые/не замужние. Так же можно увидеть, что люди в разводе, либо вдовцы чаще всего являются добросовестными заёмщиками. А люди в гражданском браке часто могут оставаться должниками.

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [27]:
#income1 = data.groupby('total_income_category')['debt'].sum() # число должников
#income2 = data.groupby('total_income_category')['debt'].count() # число займов
#income_debt_mean = income1 / income2  # поиск конверсии
#print(income1)
#print(income2)
#print(income_debt_mean.apply('{:.2%}'.format)) 

income_debt_mean = data.pivot_table(index=['total_income_category'], values='debt', aggfunc=['count', 'sum', 'mean'])
income_debt_mean.columns = ['Всего кредитополучателей', 'Всего должников', 'Доля должников']
income_debt_mean['Доля должников'] = income_debt_mean['Доля должников']*100
income_debt_mean

# Ваш код будет здесь. Вы можете создавать новые ячейки.

,Всего кредитополучателей,Всего должников,Доля должников
total_income_category,,,
A,25,2,8.000000
B,5014,354,7.060231
C,15921,1353,8.498210
D,349,21,6.017192
E,22,2,9.090909


**Вывод:** Да, зависимость есть. Но здесь крайне сложно анализировать с такими маленькими числами. Можно сказать, что чем уровень дохода больше - тем чаще возвращают кредит в срок. Это можно сказать по показателям 'C' и 'B'. Однако показатель 'D' может оспорить данное суждение, его можно принять во внимание, но слишкмо уж заёмщиков с таким уровнем дохода для анализа.

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [28]:
#category1 = data.groupby('purpose_category')['debt'].sum() # число должников
#category2 = data.groupby('purpose_category')['debt'].count() # число займов
#category_debt_mean = category1 / category2  # поиск конверсии
#print(category1)
#print(category2)
#print(category_debt_mean.apply('{:.2%}'.format)) 

category_debt_mean = data.pivot_table(index=['purpose_category'], values='debt', aggfunc=['count', 'sum', 'mean'])
category_debt_mean.columns = ['Всего кредитополучателей', 'Всего должников', 'Доля должников']
category_debt_mean['Доля должников'] = category_debt_mean['Доля должников']*100
category_debt_mean.sort_values(by = 'Доля должников', ascending = True)



# Ваш код будет здесь. Вы можете создавать новые ячейки.

,Всего кредитополучателей,Всего должников,Доля должников
purpose_category,,,
операции с недвижимостью,10751,780,7.255139
проведение свадьбы,2313,183,7.911803
получение образования,3988,369,9.252758
операции с автомобилем,4279,400,9.347978


**Вывод:** Большинство выплаченных займов приходится на недвижимость, причем в категории недвижимости больше всего показателей, что является более точным выводом. Следом идёт категория с займом на свадьбу, данный займ так же чаще выплачивают. Ну а категории с автомобилем и образованием имеют больше всего должников.

<div class="alert alert-success">
<b>Комментарий ревьюера №2</b> 
    
Хорошо
</div>

#### 3.5 Приведите возможные причины появления пропусков в исходных данных.

*Ответ:* Причин может быть множество. Например пользователи не внесли данные о себе и тем самым образовались пропуски. Или же какие то данные могли не подтянуться. При сборке данных в таблицу аналитик мог так же совершить ошибку и случайно завести множество пропусков.
Пропуски так же могут быть сделаны умышлено, для приведения пропусков к единому формату.
Так же ответственный за таблицу может сам выбрать вид пропусков(умышленных).

<div class="alert alert-success">
<b>Комментарий ревьюера №1</b> 
    
Тут важно отметить человеческий и технический факторы. Клиенты могли не предоставить данные. Возможно, эту информацию не требуют в некоторых случаях. Проблемы могли возникнуть на этапе загрузки и на этапе выгрузки данных. Неправильный запрос, объединение таблиц также может быть причиной.
</div>

#### 3.6 Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* Медиана, как и среднее арифмитическое необходимо для оценки типичных значений выборки.
Медиана будет являться более точным показателем для количественных переменных. Однако её так же лучше использовать под ситуацию. 
Медианное значение, это значение, которое является ровно серединой между элементами, так как одна часть больше, а другая меньше. Это более точные значения, чем среднее арифмитическое, которое ищется путём суммы элементов, которые делятся на их количество.

Дополнение. Запомнил, это более чёткое выражение :)
Да, действительно, если есть значения, которые сильно выделяются из общей выборки, то медиана будет более точной.

<div class="alert alert-danger">
<b>Комментарий ревьюера №1</b> 
    
Среднее и медиана различаются, если в выборке присутствуют выбросы (значения, выделяющиеся из общей выборки). Медиана позволяет нивелировать их влияние.
</div>

### Шаг 4: общий вывод.

Напишите ваш общий вывод.
В данном проекте провёл предобработку, обработку, изменение и удаление данных, а так же провёл необходимый анализ данных и сделал выводы.
По итогу проекта можно сказать, что большинство заёмщиков не имеет детей, они так же чаще выплачивают долги. Займы в основном берут женатые/замужние люди, и среди них не особо много должников. Берут займы люди чаще с категорией уровня дохода 'C' и 'B'. Ну и так же люди чаще всего берут займы для недвижимости, они так же чаще всего и выплачивают долг за займ.

Дополнение: Действительно, в категории 'B'доля должников меньше, чем в категории 'C'.

Рекомендация банку-заказчику.
Наиболее рискованные клиенты для банка:
Имеющие 1-2 детей. 
Со средним уровнем дохода категории 'C'. Так как здесь самая большая выборка среди заёмщиков и весьма большая доля должников. У категории выше 'B'- доля должников куда меньше.
Ну и так же самыми рискованными категориями на заём являются - получение образования и операции с автомобилем.

<div class="alert alert-danger">
<b>Комментарий ревьюера №1</b> 
    
Можно отметить, что доля должников в категории В ниже, чем в категории С. Также можно добавить рекомендации банку-заказчику, описать портрет наиболее рискованного клиента
</div>

<div style="border:solid purple 2px; padding: 20px"> 
<b> <font color='purple' >   
    
В целом, работа хорошая. У тебя отличные аналитические навыки, но остались небольшие доработки. Доделывай и присылай на повторную проверку.
        
Жду твоих исправлений 😊
</font></b>
</div>

<div style="border:solid purple 2px; padding: 20px"> 
<b> <font color='purple' >   
Отлично! Критических замечаний больше нет. Проект принят! 👏 Поздравляю и желаю удачи!
</font></b>
</div>